In [3]:
import cv2
import os

# Function to Extract Frames from Video
def extract_frames_from_videos(video_paths, output_base_folder="../outputs/", frame_rate=10):
    # Create base output directory if it doesn't exist
    os.makedirs(output_base_folder, exist_ok=True)

    for video_path in video_paths:
        # Extract video name without extension to name the output folder
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        output_folder = os.path.join(output_base_folder, f"frames_{video_name}")
        os.makedirs(output_folder, exist_ok=True)

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Cannot open video file: {video_path}")
            continue

        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print(f"End of video or error reading frame {frame_count} from {video_path}.")
                break

            if frame_count % frame_rate == 0:
                frame_name = os.path.join(output_folder, f"frame_{frame_count}.jpg")
                print(f"Saving frame {frame_count} as {frame_name}")
                success = cv2.imwrite(frame_name, frame)
                if success:
                    print(f"Saved: {frame_name}")
                else:
                    print(f"Failed to save: {frame_name}")

            frame_count += 1

        cap.release()
        print(f"Frames from {video_path} saved in {output_folder}")

# Example usage
video_paths = [
    r"../data/videos/video1.mp4",
    r"../data/videos/video2.mp4",
    r"../data/videos/video3.mp4",
    r"../data/videos/video4.mp4"
]

extract_frames_from_videos(video_paths)


Saving frame 0 as ../outputs/frames_video1\frame_0.jpg
Saved: ../outputs/frames_video1\frame_0.jpg
Saving frame 10 as ../outputs/frames_video1\frame_10.jpg
Saved: ../outputs/frames_video1\frame_10.jpg
Saving frame 20 as ../outputs/frames_video1\frame_20.jpg
Saved: ../outputs/frames_video1\frame_20.jpg
Saving frame 30 as ../outputs/frames_video1\frame_30.jpg
Saved: ../outputs/frames_video1\frame_30.jpg
Saving frame 40 as ../outputs/frames_video1\frame_40.jpg
Saved: ../outputs/frames_video1\frame_40.jpg
Saving frame 50 as ../outputs/frames_video1\frame_50.jpg
Saved: ../outputs/frames_video1\frame_50.jpg
Saving frame 60 as ../outputs/frames_video1\frame_60.jpg
Saved: ../outputs/frames_video1\frame_60.jpg
Saving frame 70 as ../outputs/frames_video1\frame_70.jpg
Saved: ../outputs/frames_video1\frame_70.jpg
Saving frame 80 as ../outputs/frames_video1\frame_80.jpg
Saved: ../outputs/frames_video1\frame_80.jpg
Saving frame 90 as ../outputs/frames_video1\frame_90.jpg
Saved: ../outputs/frames_vid

In [4]:
from ultralytics import YOLO

# Load YOLO Model
model = YOLO("models/yolo11n.pt")

# Vehicle Detection Function
def detect_vehicles(image_path):
    results = model(image_path)
    vehicles = len(results[0].boxes)  # Count bounding boxes
    return vehicles

# Test Detection on One Frame
vehicle_count = detect_vehicles("../outputs/frames_video1/frame_100.jpg")
print(f"Detected {vehicle_count} vehicles in frame_100.jpg")


image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_100.jpg: 384x640 7 persons, 15 cars, 4 motorcycles, 6 trucks, 318.3ms
Speed: 20.2ms preprocess, 318.3ms inference, 32.2ms postprocess per image at shape (1, 3, 384, 640)
Detected 32 vehicles in frame_100.jpg


In [5]:
import os
import pandas as pd

def generate_metrics_for_folders(frame_folders, output_folder="../outputs/"):
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists
    
    for frame_folder in frame_folders:
        folder_name = os.path.basename(frame_folder.rstrip("/"))
        csv_filename = f"{folder_name}_metrics.csv"  # Generate CSV file name
        csv_path = os.path.join(output_folder, csv_filename)
        
        print(f"Processing folder: {frame_folder}")
        metrics = []
        
        for frame in os.listdir(frame_folder):
            frame_path = os.path.join(frame_folder, frame)
            
            # Detect vehicles in the frame (replace with your actual detection logic)
            vehicle_count = detect_vehicles(frame_path)
            metrics.append({"Frame": frame, "Vehicle_Count": vehicle_count})
        
        # Save metrics to CSV
        metrics_df = pd.DataFrame(metrics)
        metrics_df.to_csv(csv_path, index=False)
        
        print(f"Metrics saved to {csv_path}")

# Example Usage
frame_folders = [
    "../outputs/frames_video1/",
    "../outputs/frames_video2/",
    "../outputs/frames_video3/",
    "../outputs/frames_video4/"
]

generate_metrics_for_folders(frame_folders)


Processing folder: ../outputs/frames_video1/

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_0.jpg: 384x640 3 persons, 16 cars, 2 motorcycles, 467.3ms
Speed: 26.3ms preprocess, 467.3ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_10.jpg: 384x640 4 persons, 17 cars, 6 motorcycles, 1 bus, 1 truck, 398.8ms
Speed: 4.0ms preprocess, 398.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_100.jpg: 384x640 7 persons, 15 cars, 4 motorcycles, 6 trucks, 143.1ms
Speed: 3.0ms preprocess, 143.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_1000.jpg: 480x640

In [6]:
import os
import pandas as pd
import math
import cv2
from ultralytics import YOLO

# Constants for speed calculation
FRAME_RATE = 10  # frames per second (adjust based on your video properties)
TIME_INTERVAL = 1 / FRAME_RATE  # seconds
SCALING_FACTOR = 0.05  # meters per pixel (adjust based on calibration)

# Load YOLO Model
model = YOLO("models/yolo11n.pt")

# Vehicle Detection Function
def detect_vehicles(image_path):
    results = model(image_path)
    vehicles = []
    for box in results[0].boxes:
        # Calculate center of bounding box
        x_center = (box.xyxy[0][0] + box.xyxy[0][2]) / 2
        y_center = (box.xyxy[0][1] + box.xyxy[0][3]) / 2
        vehicles.append((x_center.item(), y_center.item()))  # Convert to Python floats
    print(f"Frame: {image_path}, Detected Vehicles: {vehicles}")
    return vehicles

# Match positions between consecutive frames
def match_positions(previous_positions, current_positions):
    matched_positions = []
    for prev in previous_positions:
        if not current_positions:
            break
        # Find the closest match in current positions
        closest_match = min(current_positions, key=lambda curr: math.sqrt((curr[0] - prev[0])**2 + (curr[1] - prev[1])**2))
        matched_positions.append(closest_match)
    return matched_positions

# Calculate average speeds for all frames
def calculate_speeds_in_kmph(frames_folder, scaling_factor, time_interval):
    frames = sorted(os.listdir(frames_folder))  # Ensure frames are processed in order
    previous_positions = []  # Track positions of vehicles in previous frame
    speeds = []  # Store average speeds for each frame

    for i, frame in enumerate(frames):
        frame_path = os.path.join(frames_folder, frame)
        current_positions = detect_vehicles(frame_path)

        if i > 0:  # Calculate speed only after the first frame
            matched_positions = match_positions(previous_positions, current_positions)
            frame_speeds = []

            for prev, curr in zip(previous_positions, matched_positions):
                dx = curr[0] - prev[0]
                dy = curr[1] - prev[1]
                pixel_displacement = math.sqrt(dx**2 + dy**2)
                real_displacement = pixel_displacement * scaling_factor  # meters
                speed_mps = real_displacement / time_interval  # meters per second
                speed_kmph = speed_mps * 3.6  # Convert to km/h

                frame_speeds.append(speed_kmph)

            # Calculate average speed for the frame
            avg_speed = sum(frame_speeds) / len(frame_speeds) if frame_speeds else 0
            speeds.append(avg_speed)
        else:
            speeds.append(0)  # No speed for the first frame

        previous_positions = current_positions  # Update for next iteration

    return speeds

# Update CSV file with speeds
def update_csv_with_speeds(csv_path, frames_folder):
    # Load the existing metrics CSV file
    metrics_df = pd.read_csv(csv_path)
    print(f"Loaded metrics from {csv_path}")

    # Calculate speeds
    avg_speeds_kmph = calculate_speeds_in_kmph(frames_folder, SCALING_FACTOR, TIME_INTERVAL)

    # Add a new column for average speeds (km/h)
    metrics_df['Average_Speed_kmph'] = avg_speeds_kmph

    # Save the updated CSV file
    updated_csv_path = csv_path
    metrics_df.to_csv(updated_csv_path, index=False)
    print(f"Updated metrics saved to {updated_csv_path}")

# Main Execution
if __name__ == "__main__":
    # Paths for frame folders and corresponding CSV files
    frame_folders_and_csvs = [
        {"frames_folder": "../outputs/frames_video1/", "metrics_csv": "../outputs/frames_video1_metrics.csv"},
        {"frames_folder": "../outputs/frames_video2/", "metrics_csv": "../outputs/frames_video2_metrics.csv"},
        {"frames_folder": "../outputs/frames_video3/", "metrics_csv": "../outputs/frames_video3_metrics.csv"},
        {"frames_folder": "../outputs/frames_video4/", "metrics_csv": "../outputs/frames_video4_metrics.csv"},
    ]

    # Iterate over each frame folder and update its respective CSV
    for item in frame_folders_and_csvs:
        frames_folder = item["frames_folder"]
        metrics_csv_path = item["metrics_csv"]

        # Ensure the frames folder and CSV file exist
        if not os.path.exists(frames_folder):
            print(f"Error: Frames folder '{frames_folder}' not found.")
            continue
        if not os.path.exists(metrics_csv_path):
            print(f"Error: Metrics CSV file '{metrics_csv_path}' not found.")
            continue

        # Update the CSV with speed calculations
        update_csv_with_speeds(metrics_csv_path, frames_folder)


Loaded metrics from ../outputs/frames_video1_metrics.csv

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_0.jpg: 384x640 3 persons, 16 cars, 2 motorcycles, 146.0ms
Speed: 4.0ms preprocess, 146.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Frame: ../outputs/frames_video1/frame_0.jpg, Detected Vehicles: [(327.0728759765625, 351.6009521484375), (532.3650512695312, 324.6290283203125), (132.3552703857422, 347.1717834472656), (407.00128173828125, 303.3850402832031), (474.090576171875, 386.1679382324219), (65.76460266113281, 311.52825927734375), (303.1932373046875, 395.7763977050781), (141.32717895507812, 252.89210510253906), (194.85369873046875, 365.896240234375), (335.5382080078125, 267.05975341796875), (104.49498748779297, 417.80926513671875), (430.0935974121094, 225.31097412109375), (399.069091796875, 181.80056762695312), (623.7138671875, 276.40673828125), (24.142154693603516, 415.27685546875), (514.915039

In [7]:
import os
import pandas as pd
import math
from ultralytics import YOLO

# Constants
SCALING_FACTOR = 0.05  # meters per pixel (adjust based on calibration)

# Load YOLO Model
model = YOLO("models/yolo11n.pt")

# Vehicle Detection Function
def detect_vehicles(image_path):
    results = model(image_path)
    vehicles = []
    for box in results[0].boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Bounding box coordinates
        vehicles.append((x1.item(), y1.item(), x2.item(), y2.item()))  # Convert to Python floats
    print(f"Frame: {image_path}, Detected Vehicles: {vehicles}")
    return vehicles

# Calculate Queue Length
def calculate_queue_length(vehicles, axis="y"):
    """
    Calculate queue length based on vehicle bounding boxes.
    
    Args:
        vehicles (list): List of bounding boxes [(x1, y1, x2, y2), ...].
        axis (str): 'x' for horizontal queue or 'y' for vertical queue.
        
    Returns:
        float: Queue length in meters.
    """
    if not vehicles:
        return 0  # No vehicles detected, queue length is 0
    
    if axis == "x":  # Horizontal queue
        queue_positions = [((x1 + x2) / 2) for x1, y1, x2, y2 in vehicles]  # X centers
    elif axis == "y":  # Vertical queue
        queue_positions = [((y1 + y2) / 2) for x1, y1, x2, y2 in vehicles]  # Y centers
    else:
        raise ValueError("Invalid axis. Use 'x' for horizontal or 'y' for vertical.")

    # Sort vehicle positions along the queue axis
    queue_positions.sort()

    # Calculate total pixel distance
    total_pixel_distance = 0
    for i in range(1, len(queue_positions)):
        total_pixel_distance += abs(queue_positions[i] - queue_positions[i - 1])

    # Convert to meters
    queue_length_meters = total_pixel_distance * SCALING_FACTOR
    return queue_length_meters

# Update CSV with Queue Length
def update_csv_with_queue_length(csv_path, frames_folder, axis="y"):
    # Load the existing metrics CSV file
    metrics_df = pd.read_csv(csv_path)
    print(f"Loaded metrics from {csv_path}")

    # Calculate queue lengths for all frames
    queue_lengths = []
    for frame in sorted(os.listdir(frames_folder)):
        frame_path = os.path.join(frames_folder, frame)
        vehicles = detect_vehicles(frame_path)
        queue_length = calculate_queue_length(vehicles, axis=axis)
        queue_lengths.append(queue_length)
        print(f"Frame: {frame}, Queue Length: {queue_length:.2f} meters")

    # Add queue length column to the dataframe
    metrics_df['Queue_Length_meters'] = queue_lengths

    # Save the updated CSV file
    updated_csv_path = csv_path
    metrics_df.to_csv(updated_csv_path, index=False)
    print(f"Updated metrics saved to {updated_csv_path}")

# Main Execution
if __name__ == "__main__":
    # Paths for frame folders and corresponding CSV files
    frame_folders_and_csvs = [
        {"frames_folder": "../outputs/frames_video1/", "metrics_csv": "../outputs/frames_video1_metrics.csv"},
        {"frames_folder": "../outputs/frames_video2/", "metrics_csv": "../outputs/frames_video2_metrics.csv"},
        {"frames_folder": "../outputs/frames_video3/", "metrics_csv": "../outputs/frames_video3_metrics.csv"},
        {"frames_folder": "../outputs/frames_video4/", "metrics_csv": "../outputs/frames_video4_metrics.csv"},
    ]

    # Iterate over each frame folder and update its respective CSV
    for item in frame_folders_and_csvs:
        frames_folder = item["frames_folder"]
        metrics_csv_path = item["metrics_csv"]

        # Ensure the frames folder and CSV file exist
        if not os.path.exists(frames_folder):
            print(f"Error: Frames folder '{frames_folder}' not found.")
            continue
        if not os.path.exists(metrics_csv_path):
            print(f"Error: Metrics CSV file '{metrics_csv_path}' not found.")
            continue

        # Update the CSV with queue length calculations
        # Choose 'x' for horizontal queues or 'y' for vertical queues
        update_csv_with_queue_length(metrics_csv_path, frames_folder, axis="y")


Loaded metrics from ../outputs/frames_video1_metrics.csv

image 1/1 c:\Users\Admin\Desktop\Traffic Management System\ML Model\notebooks\..\outputs\frames_video1\frame_0.jpg: 384x640 3 persons, 16 cars, 2 motorcycles, 1887.5ms
Speed: 22.7ms preprocess, 1887.5ms inference, 38.3ms postprocess per image at shape (1, 3, 384, 640)
Frame: ../outputs/frames_video1/frame_0.jpg, Detected Vehicles: [(271.8935241699219, 306.1990661621094, 382.2522277832031, 397.0028381347656), (488.0491027832031, 283.3375244140625, 576.6810302734375, 365.9205627441406), (74.78852081298828, 294.1481018066406, 189.92201232910156, 400.1954650878906), (363.7078857421875, 265.6563415527344, 450.294677734375, 341.1137390136719), (409.9002685546875, 340.6671447753906, 538.2808837890625, 431.6687316894531), (8.471732139587402, 274.317138671875, 123.05747985839844, 348.7393798828125), (242.86117553710938, 359.85498046875, 363.5252685546875, 431.69781494140625), (100.08544921875, 219.97378540039062, 182.5689239501953, 285.8

In [8]:
import pandas as pd
import os

# Congestion Classification Function
def classify_congestion(queue_length):
    """
    Classify congestion levels based on queue length.
    
    Args:
        queue_length (float): Queue length in meters.
        
    Returns:
        str: Congestion level ('Low', 'Medium', 'High').
    """
    if queue_length <= 10:
        return "Low"
    elif 10 < queue_length <= 20:
        return "Medium"
    else:
        return "High"

# Update CSV with Congestion Levels
def update_csv_with_congestion_levels(csv_path):
    # Load the existing metrics CSV file
    metrics_df = pd.read_csv(csv_path)
    print(f"Loaded metrics from {csv_path}")

    # Ensure 'Queue_Length_meters' column exists
    if 'Queue_Length_meters' not in metrics_df.columns:
        print("Error: 'Queue_Length_meters' column not found in the CSV file.")
        return

    # Classify congestion levels for each frame
    metrics_df['Congestion_Level'] = metrics_df['Queue_Length_meters'].apply(classify_congestion)

    # Save the updated CSV file
    updated_csv_path = csv_path
    metrics_df.to_csv(updated_csv_path, index=False)
    print(f"Updated metrics with congestion levels saved to {updated_csv_path}")

# Main Execution
if __name__ == "__main__":
    # Paths to the metrics CSV files
    metrics_csv_files = [
        "../outputs/frames_video1_metrics.csv",
        "../outputs/frames_video2_metrics.csv",
        "../outputs/frames_video3_metrics.csv",
        "../outputs/frames_video4_metrics.csv",
    ]

    # Process each CSV file
    for csv_path in metrics_csv_files:
        if not os.path.exists(csv_path):
            print(f"Error: Metrics CSV file '{csv_path}' not found.")
            continue
        update_csv_with_congestion_levels(csv_path)


Loaded metrics from ../outputs/frames_video1_metrics.csv
Updated metrics with congestion levels saved to ../outputs/frames_video1_metrics.csv
Loaded metrics from ../outputs/frames_video2_metrics.csv
Updated metrics with congestion levels saved to ../outputs/frames_video2_metrics.csv
Loaded metrics from ../outputs/frames_video3_metrics.csv
Updated metrics with congestion levels saved to ../outputs/frames_video3_metrics.csv
Loaded metrics from ../outputs/frames_video4_metrics.csv
Updated metrics with congestion levels saved to ../outputs/frames_video4_metrics.csv


In [9]:
import pandas as pd
import os

# Traffic Density Calculation Function
def calculate_traffic_density(vehicle_count, queue_length):
    """
    Calculate traffic density as vehicles per meter.
    
    Args:
        vehicle_count (int): Number of vehicles detected in the frame.
        queue_length (float): Queue length in meters.
        
    Returns:
        float: Traffic density (vehicles per meter).
    """
    if queue_length == 0:  # Avoid division by zero
        return 0
    return vehicle_count / queue_length

# Update CSV with Traffic Density
def update_csv_with_traffic_density(csv_path):
    # Load the existing metrics CSV file
    metrics_df = pd.read_csv(csv_path)
    print(f"Loaded metrics from {csv_path}")

    # Ensure required columns exist
    if 'Vehicle_Count' not in metrics_df.columns or 'Queue_Length_meters' not in metrics_df.columns:
        print("Error: Required columns 'Vehicle_Count' and 'Queue_Length_meters' are missing.")
        return

    # Calculate traffic density for each frame
    metrics_df['Traffic_Density_vehicles_per_meter'] = metrics_df.apply(
        lambda row: calculate_traffic_density(row['Vehicle_Count'], row['Queue_Length_meters']),
        axis=1
    )

    # Save the updated CSV file
    updated_csv_path = csv_path
    metrics_df.to_csv(updated_csv_path, index=False)
    print(f"Updated metrics with traffic density saved to {updated_csv_path}")

# Main Execution
if __name__ == "__main__":
    # Paths to the metrics CSV files
    metrics_csv_files = [
        "../outputs/frames_video1_metrics.csv",
        "../outputs/frames_video2_metrics.csv",
        "../outputs/frames_video3_metrics.csv",
        "../outputs/frames_video4_metrics.csv",
    ]

    # Process each CSV file
    for csv_path in metrics_csv_files:
        if not os.path.exists(csv_path):
            print(f"Error: Metrics CSV file '{csv_path}' not found.")
            continue
        update_csv_with_traffic_density(csv_path)


Loaded metrics from ../outputs/frames_video1_metrics.csv


Updated metrics with traffic density saved to ../outputs/frames_video1_metrics.csv
Loaded metrics from ../outputs/frames_video2_metrics.csv
Updated metrics with traffic density saved to ../outputs/frames_video2_metrics.csv
Loaded metrics from ../outputs/frames_video3_metrics.csv
Updated metrics with traffic density saved to ../outputs/frames_video3_metrics.csv
Loaded metrics from ../outputs/frames_video4_metrics.csv
Updated metrics with traffic density saved to ../outputs/frames_video4_metrics.csv
